# Form the NB Regression Random Intercept Model: Simulated set

It kinda works except I'm not sure why the $\beta$ is the wrong sign.. 

When we have 2 variance components it takes hella long, when we just have one it works fast.

In [1]:
using DataFrames, Random, GLM, GLMCopula, Revise
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!

Random.seed!(1235)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.9
variance_component_2 = 0.1

r = 1
p = 0.2
μ = r * p * inv(1-p)
# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Array{Array{Float64,1},1}:
 [5.0, 2.0, 15.0, 13.0, 18.0]
 [1.0, 13.0, 20.0, 4.0, 32.0]
 [6.0, 3.0, 0.0, 2.0, 0.0]
 [16.0, 1.0, 0.0, 10.0, 8.0]
 [5.0, 16.0, 1.0, 7.0, 3.0]
 [3.0, 0.0, 3.0, 1.0, 2.0]
 [25.0, 3.0, 0.0, 15.0, 4.0]
 [8.0, 5.0, 0.0, 20.0, 2.0]
 [0.0, 5.0, 19.0, 0.0, 7.0]
 [5.0, 3.0, 11.0, 2.0, 17.0]
 [7.0, 1.0, 1.0, 0.0, 0.0]
 [8.0, 7.0, 1.0, 4.0, 9.0]
 [5.0, 19.0, 2.0, 8.0, 0.0]
 ⋮
 [1.0, 3.0, 6.0, 4.0, 22.0]
 [1.0, 0.0, 7.0, 1.0, 0.0]
 [0.0, 1.0, 5.0, 0.0, 1.0]
 [1.0, 3.0, 5.0, 6.0, 5.0]
 [5.0, 6.0, 8.0, 1.0, 32.0]
 [0.0, 0.0, 0.0, 1.0, 0.0]
 [7.0, 17.0, 6.0, 6.0, 4.0]
 [1.0, 2.0, 1.0, 3.0, 5.0]
 [2.0, 12.0, 1.0, 1.0, 13.0]
 [0.0, 0.0, 3.0, 3.0, 2.0]
 [1.0, 4.0, 3.0, 0.0, 1.0]
 [9.0, 6.0, 0.0, 1.0, 1.0]

In [2]:
d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end
gcm = GLMCopulaVCModel(gcs);

In [3]:
initialize_model!(gcm)
@show gcm.β

gcm.β = [1.6613963855412255]


1-element Array{Float64,1}:
 1.6613963855412255

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [4]:
fill!(gcm.Σ, 1.0)
update_Σ!(gcm)
@show gcm.Σ
GLMCopula.loglikelihood!(gcm, true, true)

gcm.Σ = [0.27630005229752985, 8.16239668111237e-5]


-136522.99616499353

In [5]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 500, hessian_approximation = "exact"))

gcm.Σ = [0.27629971943625786, 8.042910708116454e-5]
gcm.Σ = [0.2762993914904858, 7.925183429940428e-5]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper

gcm.Σ = [0.9056543779017305, 0.11232347936536051]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  50  1.3537980e+05 0.00e+00 3.80e-07  -8.6 2.94e-08    -  1.00e+00 1.00e+00w  1
gcm.Σ = [0.9056543910381998, 0.11232347846295936]
  51  1.3537980e+05 0.00e+00 3.65e-07  -8.6 5.39e-10    -  1.00e+00 1.00e+00w  1
gcm.Σ = [0.9056544006343334, 0.11232347808814978]
  52  1.3537980e+05 0.00e+00 2.62e-07  -8.6 5.19e-10    -  1.00e+00 1.00e+00w  1
gcm.Σ = [0.9056543682028455, 0.11232348130411661]
  53  1.3537980e+05 0.00e+00 2.07e-05  -8.6 3.72e-10    -  1.00e+00 4.77e-07f 21
gcm.Σ = [0.9056543661475474, 0.11232348085311868]
  54  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.95e-08    -  1.00e+00 1.19e-07f 24
gcm.Σ = [0.9056543660494472, 0.11232348063719962]
  55  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 2.38e-07f 23
gcm.Σ = [0.9056543659459546, 0.11232348056056439]
  56  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 2.38e-07f 23
gc

gcm.Σ = [0.9056543656936648, 0.11232348039448972]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 110  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 2.38e-07f 23
gcm.Σ = [0.9056543652534844, 0.11232348039205367]
 111  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 5.96e-08f 25
gcm.Σ = [0.905654365210877, 0.11232348036816804]
 112  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 5.96e-08f 25
gcm.Σ = [0.9056543649604721, 0.11232348032236962]
 113  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 2.98e-08f 26
gcm.Σ = [0.9056543649195594, 0.11232348029583217]
 114  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 2.98e-08f 26
gcm.Σ = [0.9056543765121539, 0.11232347920513829]
 115  1.3537980e+05 0.00e+00 3.92e-07  -8.6 2.94e-08    -  1.00e+00 1.00e+00w  1
gcm.Σ = [0.9056543899568755, 0.11232347828134426]
 116  1.3537980e+05 0.00e+00 3.74e-07  -8.6 5.57e-10    -  1.00e+00 1.00e+00w  1
gcm

gcm.Σ = [0.9056543655151365, 0.11232348075951264]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170  1.3537980e+05 0.00e+00 2.07e-05  -8.6 3.83e-10    -  1.00e+00 3.73e-09f 28
gcm.Σ = [0.9056543645346554, 0.11232348032837726]
 171  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543642983972, 0.11232348016484552]
 172  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543642193737, 0.1123234801092002]
 173  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641926488, 0.11232348009033542]
 174  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641835972, 0.11232348008394524]
 175  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.905654364180451, 0.11232348008176238]
 176  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.

gcm.Σ = [0.9056543641790882, 0.11232348008079318]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 230  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641789576, 0.11232348008070077]
 231  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543759092299, 0.11232347899560893]
 232  1.3537980e+05 0.00e+00 3.95e-07  -8.6 2.94e-08    -  1.00e+00 1.00e+00w  1
gcm.Σ = [0.9056543894534527, 0.11232347808126004]
 233  1.3537980e+05 0.00e+00 3.76e-07  -8.6 5.62e-10    -  1.00e+00 1.00e+00w  1
gcm.Σ = [0.9056543993524482, 0.1123234777086425]
 234  1.3537980e+05 0.00e+00 2.70e-07  -8.6 5.34e-10    -  1.00e+00 1.00e+00w  1
gcm.Σ = [0.9056543655151531, 0.11232348075948134]
 235  1.3537980e+05 0.00e+00 2.07e-05  -8.6 3.83e-10    -  1.00e+00 3.73e-09f 28
gcm.Σ = [0.9056543645346573, 0.11232348032837493]
 236  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm

gcm.Σ = [0.9056543642193744, 0.11232348010919965]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 290  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641926488, 0.1123234800903354]
 291  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641835972, 0.11232348008394524]
 292  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.905654364180451, 0.11232348008176238]
 293  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641794318, 0.11232348008102513]
 294  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641790882, 0.11232348008079318]
 295  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641789576, 0.11232348008070077]
 296  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.

gcm.Σ = [0.9056543894534527, 0.11232347808126004]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 350  1.3537980e+05 0.00e+00 3.76e-07  -8.6 5.62e-10    -  1.00e+00 1.00e+00w  1
gcm.Σ = [0.9056543993524482, 0.1123234777086425]
 351  1.3537980e+05 0.00e+00 2.70e-07  -8.6 5.34e-10    -  1.00e+00 1.00e+00w  1
gcm.Σ = [0.9056543655151531, 0.11232348075948134]
 352  1.3537980e+05 0.00e+00 2.07e-05  -8.6 3.83e-10    -  1.00e+00 3.73e-09f 28
gcm.Σ = [0.9056543645346573, 0.11232348032837493]
 353  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543642983567, 0.11232348016483845]
 354  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543642193744, 0.11232348010919965]
 355  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641926488, 0.1123234800903354]
 356  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.

gcm.Σ = [0.905654364180451, 0.11232348008176238]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 410  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641794318, 0.11232348008102513]
 411  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641790882, 0.11232348008079318]
 412  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641789576, 0.11232348008070077]
 413  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543759092299, 0.11232347899560893]
 414  1.3537980e+05 0.00e+00 3.95e-07  -8.6 2.94e-08    -  1.00e+00 1.00e+00w  1
gcm.Σ = [0.9056543894534527, 0.11232347808126004]
 415  1.3537980e+05 0.00e+00 3.76e-07  -8.6 5.62e-10    -  1.00e+00 1.00e+00w  1
gcm.Σ = [0.9056543993524482, 0.1123234777086425]
 416  1.3537980e+05 0.00e+00 2.70e-07  -8.6 5.34e-10    -  1.00e+00 1.00e+00w  1
gcm.

gcm.Σ = [0.9056543645346573, 0.11232348032837493]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 470  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543642983567, 0.11232348016483845]
 471  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543642193744, 0.11232348010919965]
 472  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641926488, 0.1123234800903354]
 473  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641835972, 0.11232348008394524]
 474  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.905654364180451, 0.11232348008176238]
 475  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.Σ = [0.9056543641794318, 0.11232348008102513]
 476  1.3537980e+05 0.00e+00 2.07e-05  -8.6 2.94e-08    -  1.00e+00 3.73e-09f 29
gcm.

┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/sarahji/.julia/dev/GLMCopula.jl/src/fit_new.jl:17


In [6]:
@show β_true

@show gcm.β
@show gcm.Σ
@show gcm.∇β
@show GLMCopula.loglikelihood!(gcm, true, true);

β_true = -1.3862943611198906
gcm.β = [1.381907542656039]
gcm.Σ = [0.9056543641835972, 0.11232348008394524]
gcm.∇β = [-0.0013352918968153604]
GLMCopula.loglikelihood!(gcm, true, true) = -135379.80211234628


#### With Just one Variance Component 

In [7]:
# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.9

r = 1
p = 0.2
μ = r * p * inv(1-p)
# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Array{Array{Float64,1},1}:
 [0.0, 0.0, 5.0, 0.0, 1.0]
 [40.0, 5.0, 0.0, 12.0, 0.0]
 [3.0, 0.0, 7.0, 0.0, 1.0]
 [1.0, 6.0, 0.0, 1.0, 1.0]
 [5.0, 6.0, 3.0, 1.0, 2.0]
 [1.0, 3.0, 3.0, 4.0, 1.0]
 [16.0, 8.0, 3.0, 1.0, 11.0]
 [2.0, 1.0, 0.0, 0.0, 1.0]
 [0.0, 0.0, 20.0, 1.0, 14.0]
 [15.0, 2.0, 29.0, 14.0, 0.0]
 [24.0, 4.0, 6.0, 0.0, 1.0]
 [8.0, 4.0, 3.0, 12.0, 15.0]
 [12.0, 2.0, 2.0, 3.0, 15.0]
 ⋮
 [1.0, 1.0, 4.0, 0.0, 1.0]
 [1.0, 1.0, 4.0, 5.0, 0.0]
 [12.0, 0.0, 5.0, 0.0, 1.0]
 [2.0, 0.0, 1.0, 7.0, 0.0]
 [3.0, 12.0, 8.0, 7.0, 1.0]
 [2.0, 1.0, 9.0, 1.0, 4.0]
 [6.0, 1.0, 1.0, 0.0, 0.0]
 [9.0, 0.0, 1.0, 24.0, 28.0]
 [0.0, 1.0, 0.0, 1.0, 1.0]
 [6.0, 13.0, 2.0, 0.0, 2.0]
 [0.0, 1.0, 15.0, 8.0, 4.0]
 [6.0, 1.0, 9.0, 2.0, 11.0]

In [8]:
d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end
gcm = GLMCopulaVCModel(gcs);

In [9]:
initialize_model!(gcm)
@show gcm.β
fill!(gcm.Σ, 1.0)
update_Σ!(gcm)
@show gcm.Σ
GLMCopula.loglikelihood!(gcm, true, true)

gcm.β = [1.6616925445883912]
gcm.Σ = [0.3301128865664049]


-136338.67487465

In [10]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 500, hessian_approximation = "exact"))

gcm.Σ = [0.3301118462400453]
gcm.Σ = [0.3301109506193979]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr  

In [11]:
@show β_true

@show gcm.β
@show gcm.Σ
@show gcm.∇β
@show GLMCopula.loglikelihood!(gcm, true, true);

β_true = -1.3862943611198906
gcm.β = [1.3877884846534372]
gcm.Σ = [0.9016776767499567]
gcm.∇β = [-4.543350438090954e-7]
GLMCopula.loglikelihood!(gcm, true, true) = -135131.13529616
